In [1]:
import os
import numpy as np
import xarray as xr
import earthaccess
from datetime import datetime, timedelta
import pickle
import time
import random
from tqdm import tqdm

/Users/jessecox/.pyenv/versions/3.11.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Authenticate
auth = earthaccess.login(persist=True)

In [3]:
# Parameters
bbox = (-83.62, 41.34, -82, 42.27)      # (lon_min, lat_min, lon_max, lat_max)
res = 0.01
start_date = datetime(2024, 4, 14)
end_date = datetime(2024, 4, 20)

In [4]:
# Ensure data directories exist
os.makedirs("../Data/", exist_ok=True)
os.makedirs("../Cache/", exist_ok=True)
os.makedirs("../Images/", exist_ok=True)

In [5]:
# Output grid
lat_bins = np.arange(bbox[1], bbox[3] + res, res)
lon_bins = np.arange(bbox[0], bbox[2] + res, res)
lat_centers = 0.5 * (lat_bins[:-1] + lat_bins[1:])
lon_centers = 0.5 * (lon_bins[:-1] + lon_bins[1:])
nlat, nlon = len(lat_centers), len(lon_centers)

In [6]:
# --- Retry Helpers ---
def safe_search(short_name, temporal, bounding_box, max_retries=500):
    retries = 0
    while True:
        try:
            results = earthaccess.search_data(
                short_name=short_name,
                temporal=temporal,
                bounding_box=bounding_box
            )
            return results
        except Exception as e:
            retries += 1
            if retries >= max_retries:
                print(f"Search failed after {max_retries} retries: {e}")
                return []
            wait = 5 + random.uniform(0, 3)
            print(f"Search error: {e}. Retrying in {wait:.1f} seconds...")
            time.sleep(wait)

In [7]:
def safe_download(results, directory="../Data/", max_retries=5):
    retries = 0
    while True:
        try:
            paths = earthaccess.download(results, directory)
            return paths
        except Exception as e:
            retries += 1
            if retries >= max_retries:
                print(f"Download failed after {max_retries} retries: {e}")
                return []
            wait = 5 + random.uniform(0, 3)
            print(f"Download error: {e}. Retrying in {wait:.1f} seconds...")
            time.sleep(wait)

In [8]:
# Wavelengths from a reference file (S3A or S3B, any available date)
def get_reference_wavelengths():
    satellites = ["MODISA_L2_OC", "MODIST_L2_OC"]
    ref_file = None
    search_dates = [("2024-06-01", "2024-06-05"), ("2024-05-01", "2024-06-01"), ("2024-04-01", "2024-05-01")]

    for sat in satellites:
        for temporal in search_dates:
            print(f"Trying to find wavelengths in {sat} from {temporal}...")

            search_ref = safe_search(
                short_name=sat,
                temporal=temporal,
                bounding_box=bbox,
            )
            if search_ref:
                ref_file = safe_download(search_ref, "../Data/")[0]
                try:
                    with xr.open_dataset(ref_file, group="sensor_band_parameters") as ds:
                        if "wavelength" in ds:
                            print("found wavelength")
                            wave_all = ds["wavelength"].data
                        elif "wavelength_3d" in ds:
                            print("found wavelength_3d")
                            wave_all = ds["wavelength_3d"].data
                        else:
                            raise KeyError("No 'wavelength' variable found in reference file.")
                    print(f"Found wavelengths in {ref_file} ({len(wave_all)} channels)")
                    return wave_all
                except Exception as e:
                    print(f"Failed to extract wavelengths from {ref_file}: {e}")
    raise RuntimeError("No reference files found to retrieve wavelengths.")

In [9]:
# Call it
wave_all = get_reference_wavelengths()

Trying to find wavelengths in MODISA_L2_OC from ('2024-06-01', '2024-06-05')...


QUEUEING TASKS | : 100%|████████████████████████| 8/8 [00:00<00:00, 4491.89it/s]
PROCESSING TASKS | : 100%|█████████████████████| 8/8 [00:00<00:00, 19611.01it/s]
COLLECTING RESULTS | : 100%|███████████████████| 8/8 [00:00<00:00, 78951.60it/s]

found wavelength
Found wavelengths in ../Data/AQUA_MODIS.20240601T192000.L2.OC.nc (24 channels)


In [10]:
wave_all

array([  412.,   443.,   469.,   488.,   531.,   547.,   555.,   645.,
         667.,   678.,   748.,   859.,   869.,  1240.,  1640.,  2130.,
        3750.,  3959.,  4050.,  6715.,  7325.,  8550., 11000., 12000.])

In [11]:
num_channels = len(wave_all)
print(f"Found {num_channels} channels.")

Found 24 channels.


In [12]:
# Prepare main array
total_days = (end_date - start_date).days + 1
ndarray_all = np.full((total_days, nlat, nlon, num_channels), np.nan, dtype=np.float32)

In [13]:
# Process day by day
window_size = 1
for day_idx in range(total_days):
    current_date = start_date + timedelta(days=day_idx)
    window_start = current_date - timedelta(days=window_size)
    window_end = current_date - timedelta(days=1)

    print(f"\nProcessing {current_date.date()} (composite window {window_start.date()} to {window_end.date()})")

    # Search for data from both S3A and S3B
    results_ma = safe_search(
        short_name="MODISA_L2_OC",
        temporal=(window_start.strftime("%Y-%m-%d"), window_end.strftime("%Y-%m-%d")),
        bounding_box=bbox,
    )
    
    results_mt = safe_search(
        short_name="MODIST_L2_OC",
        temporal=(window_start.strftime("%Y-%m-%d"), window_end.strftime("%Y-%m-%d")),
        bounding_box=bbox,
    )
    results = results_ma + results_mt

    if not results:
        print("No data found for this window. Skipping to next date.")
        continue

    # Download new files
    to_download = []
    for granule in results:
        granule_id = granule['umm']['DataGranule']['ArchiveAndDistributionInformation'][0]['Name']
        granule_basename = os.path.basename(granule_id)
        cache_path = f"../Cache/{granule_basename}.nc.npz"
        if not os.path.exists(cache_path):
            to_download.append(granule)

    if to_download:
        print(f"Need to download {len(to_download)} new files for this window.")
        paths = safe_download(to_download, "../Data/")
        if not paths:
            print("No files downloaded for this window. Skipping to next date.")
            continue
    else:
        print("All files already cached. Skipping download.")
        paths = [f"../Data/{os.path.basename(granule['granule_id'])}" for granule in results]

    # Initialize sum and weight arrays
    sum_all = np.zeros((num_channels, nlat, nlon))
    weight_all = np.zeros((num_channels, nlat, nlon))

    for path in paths:
        base = os.path.basename(path)
        date_str = base.split(".")[1][:8]
        file_date = datetime.strptime(date_str, "%Y%m%d")
        delta_days = (window_end - file_date).days
        print('delta_days:', delta_days)
        weight = 1

        cache_file = f"../Cache/{base}.npz"

        if not os.path.exists(cache_file):
            print(f"Processing {base} (not in cache)")
            try:
                nav = xr.open_dataset(path, group="navigation_data")
                lat = nav["latitude"].values
                lon = nav["longitude"].values



                rrs_ds = xr.open_dataset(path, group="geophysical_data")
                lat_idx_all, lon_idx_all, ch_idx_all, val_all = [], [], [], []
                print(f"Available variables in {base}:")
                print([v for v in rrs_ds.variables])
                print("Expected Rrs variable names (based on reference wavelengths):")
                print([f"Rrs_{int(round(w))}" for w in wave_all])

                for ch_idx, wl in tqdm(list(enumerate(wave_all)), desc=f"Channels in {base}", leave=False):
                    var_name = f"Rrs_{int(round(wl))}"
                    if var_name not in rrs_ds:
                        continue
                    band = rrs_ds[var_name].values
                    mask = (
                        np.isfinite(band) &
                        (lat >= bbox[1]) & (lat <= bbox[3]) &
                        (lon >= bbox[0]) & (lon <= bbox[2])
                    )
                    lat_valid = lat[mask]
                    lon_valid = lon[mask]
                    val_valid = band[mask]

                    lat_idx = np.searchsorted(lat_bins, lat_valid) - 1
                    lon_idx = np.searchsorted(lon_bins, lon_valid) - 1

                    lat_idx_all.extend(lat_idx)
                    lon_idx_all.extend(lon_idx)
                    ch_idx_all.extend([ch_idx] * len(val_valid))
                    val_all.extend(val_valid)

                np.savez_compressed(cache_file,
                                    lat_idx=np.array(lat_idx_all, dtype=np.int16),
                                    lon_idx=np.array(lon_idx_all, dtype=np.int16),
                                    ch_idx=np.array(ch_idx_all, dtype=np.int16),
                                    val=np.array(val_all, dtype=np.float32))
                print(f"Cached data to {cache_file}")

            except Exception as e:
                print(f"Failed to process {path}: {e}")
                continue
        else:
            print(f"Using cached data for {base}")

        # Load from cache
        data = np.load(cache_file)
        lat_idx = data['lat_idx']
        lon_idx = data['lon_idx']
        ch_idx = data['ch_idx']
        val = data['val']

        for j in range(len(val)):
            if 0 <= lat_idx[j] < nlat and 0 <= lon_idx[j] < nlon:
                sum_all[ch_idx[j], lat_idx[j], lon_idx[j]] += val[j] * weight
                weight_all[ch_idx[j], lat_idx[j], lon_idx[j]] += weight

    # Finalize average
    with np.errstate(invalid="ignore", divide="ignore"):
        avg_all = sum_all / weight_all
        avg_all[weight_all == 0] = np.nan

    ndarray_all[day_idx] = np.transpose(avg_all, (1, 2, 0))

    # Delete old files
    delete_date = (current_date - timedelta(days=window_size)).strftime('%Y%m%d')
    for fname in os.listdir("../Data/"):
        if delete_date in fname and fname.endswith(".nc"):
            try:
                os.remove(os.path.join("../Data/", fname))
                print(f"Deleted old file: {fname}")
            except Exception as e:
                print(f"Could not delete {fname}: {e}")


Processing 2024-04-14 (composite window 2024-04-13 to 2024-04-13)
Need to download 2 new files for this window.


QUEUEING TASKS | : 100%|████████████████████████| 2/2 [00:00<00:00, 1059.30it/s]
PROCESSING TASKS | : 100%|█████████████████████| 2/2 [00:00<00:00, 13086.75it/s]
COLLECTING RESULTS | : 100%|████████████████████| 2/2 [00:00<00:00, 8439.24it/s]


delta_days: 0
Processing AQUA_MODIS.20240413T184001.L2.OC.nc (not in cache)
Available variables in AQUA_MODIS.20240413T184001.L2.OC.nc:
['aot_869', 'angstrom', 'Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'chlor_a', 'Kd_490', 'pic', 'poc', 'ipar', 'nflh', 'par', 'l2_flags']
Expected Rrs variable names (based on reference wavelengths):
['Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'Rrs_748', 'Rrs_859', 'Rrs_869', 'Rrs_1240', 'Rrs_1640', 'Rrs_2130', 'Rrs_3750', 'Rrs_3959', 'Rrs_4050', 'Rrs_6715', 'Rrs_7325', 'Rrs_8550', 'Rrs_11000', 'Rrs_12000']


Cached data to ../Cache/AQUA_MODIS.20240413T184001.L2.OC.nc.npz
delta_days: 0
Processing TERRA_MODIS.20240413T160000.L2.OC.nc (not in cache)
Available variables in TERRA_MODIS.20240413T160000.L2.OC.nc:
['aot_869', 'angstrom', 'Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'chlor_a', 'Kd_490', 'pic', 'poc', 'ipar', 'nflh', 'par', 'l2_flags']
Expected Rrs variable names (based on reference wavelengths):
['Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'Rrs_748', 'Rrs_859', 'Rrs_869', 'Rrs_1240', 'Rrs_1640', 'Rrs_2130', 'Rrs_3750', 'Rrs_3959', 'Rrs_4050', 'Rrs_6715', 'Rrs_7325', 'Rrs_8550', 'Rrs_11000', 'Rrs_12000']


Cached data to ../Cache/TERRA_MODIS.20240413T160000.L2.OC.nc.npz
Deleted old file: TERRA_MODIS.20240413T160000.L2.OC.nc
Deleted old file: AQUA_MODIS.20240413T184001.L2.OC.nc

Processing 2024-04-15 (composite window 2024-04-14 to 2024-04-14)
Need to download 3 new files for this window.


...

Cached data to ../Cache/AQUA_MODIS.20240419T192501.L2.OC.nc.npz
delta_days: 0
Processing AQUA_MODIS.20240419T193001.L2.OC.nc (not in cache)
Available variables in AQUA_MODIS.20240419T193001.L2.OC.nc:
['aot_869', 'angstrom', 'Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'chlor_a', 'Kd_490', 'pic', 'poc', 'ipar', 'nflh', 'par', 'l2_flags']
Expected Rrs variable names (based on reference wavelengths):
['Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'Rrs_748', 'Rrs_859', 'Rrs_869', 'Rrs_1240', 'Rrs_1640', 'Rrs_2130', 'Rrs_3750', 'Rrs_3959', 'Rrs_4050', 'Rrs_6715', 'Rrs_7325', 'Rrs_8550', 'Rrs_11000', 'Rrs_12000']


Cached data to ../Cache/AQUA_MODIS.20240419T193001.L2.OC.nc.npz
delta_days: 0
Processing TERRA_MODIS.20240419T151000.L2.OC.nc (not in cache)
Available variables in TERRA_MODIS.20240419T151000.L2.OC.nc:
['aot_869', 'angstrom', 'Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'chlor_a', 'Kd_490', 'pic', 'poc', 'ipar', 'nflh', 'par', 'l2_flags']
Expected Rrs variable names (based on reference wavelengths):
['Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'Rrs_748', 'Rrs_859', 'Rrs_869', 'Rrs_1240', 'Rrs_1640', 'Rrs_2130', 'Rrs_3750', 'Rrs_3959', 'Rrs_4050', 'Rrs_6715', 'Rrs_7325', 'Rrs_8550', 'Rrs_11000', 'Rrs_12000']


Cached data to ../Cache/TERRA_MODIS.20240419T151000.L2.OC.nc.npz
delta_days: 0
Processing TERRA_MODIS.20240419T164501.L2.OC.nc (not in cache)
Available variables in TERRA_MODIS.20240419T164501.L2.OC.nc:
['aot_869', 'angstrom', 'Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'chlor_a', 'Kd_490', 'pic', 'poc', 'ipar', 'nflh', 'par', 'l2_flags']
Expected Rrs variable names (based on reference wavelengths):
['Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'Rrs_748', 'Rrs_859', 'Rrs_869', 'Rrs_1240', 'Rrs_1640', 'Rrs_2130', 'Rrs_3750', 'Rrs_3959', 'Rrs_4050', 'Rrs_6715', 'Rrs_7325', 'Rrs_8550', 'Rrs_11000', 'Rrs_12000']


Cached data to ../Cache/TERRA_MODIS.20240419T164501.L2.OC.nc.npz
delta_days: 0
Processing TERRA_MODIS.20240419T165000.L2.OC.nc (not in cache)
Available variables in TERRA_MODIS.20240419T165000.L2.OC.nc:
['aot_869', 'angstrom', 'Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'chlor_a', 'Kd_490', 'pic', 'poc', 'ipar', 'nflh', 'par', 'l2_flags']
Expected Rrs variable names (based on reference wavelengths):
['Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'Rrs_748', 'Rrs_859', 'Rrs_869', 'Rrs_1240', 'Rrs_1640', 'Rrs_2130', 'Rrs_3750', 'Rrs_3959', 'Rrs_4050', 'Rrs_6715', 'Rrs_7325', 'Rrs_8550', 'Rrs_11000', 'Rrs_12000']


Cached data to ../Cache/TERRA_MODIS.20240419T165000.L2.OC.nc.npz
Deleted old file: AQUA_MODIS.20240419T193001.L2.OC.nc
Deleted old file: TERRA_MODIS.20240419T164501.L2.OC.nc
Deleted old file: AQUA_MODIS.20240419T192501.L2.OC.nc
Deleted old file: TERRA_MODIS.20240419T151000.L2.OC.nc
Deleted old file: TERRA_MODIS.20240419T165000.L2.OC.nc


In [15]:
# Save results
np.save("../Composite_Data/composite_data_MODIS.npy", ndarray_all)
print("\nSaved full 4D composite data array to '../Composite_Data/composite_data_MODIS.npy'.")

metadata = {"wavelengths": wave_all, "lat": lat_centers, "lon": lon_centers}
with open("../Composite_Data/composite_metadata_MODIS.pkl", "wb") as f:
    pickle.dump(metadata, f)
print("Saved metadata (wavelengths, lat, lon) to '../Composite_Data/composite_metadata_MODIS.pkl'.")


Saved full 4D composite data array to '../Composite_Data/composite_data_MODIS.npy'.
Saved metadata (wavelengths, lat, lon) to '../Composite_Data/composite_metadata_MODIS.pkl'.
